In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

import Reader
import ParamsExtractor3
import DataPreprocessor
import EncodeCommand
import DeepLearningAnomalyDetection2

2024-08-05 09:05:22.776218: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 09:05:22.825463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 09:05:23.642841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_paths = ['../data/laurel_short_' + str(i) for i in range(8)]
existing_file_paths = [path for path in file_paths if os.path.exists(path)]
print(existing_file_paths)

anomalous_fp = '../data/laurel_short_anomalous'

['../data/laurel_short_0', '../data/laurel_short_1', '../data/laurel_short_2', '../data/laurel_short_3', '../data/laurel_short_4', '../data/laurel_short_5', '../data/laurel_short_6', '../data/laurel_short_7']


In [3]:
df_list = []

for f in existing_file_paths:
    df = pd.read_csv(f)
    df_list.append(df)

anomalous_df = pd.read_csv(anomalous_fp)

In [4]:
df_list[0].head(3)

,severity_scores,timedelta,suid,cap_fp,exit,n_dang,n_dang_no_cron,fp_length,col_0,col_1,...,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,enc_comm,enc_parent_comm
0,1.0,0.0,126858,0,0,2,0,92,0,0,...,0,0,0,0,1,0,0,0,1,39
1,1.0,0.0,0,0,0,0,0,38,0,0,...,0,0,0,0,1,0,0,0,2,82
2,1.0,0.0,0,0,0,0,0,28,0,0,...,0,0,0,0,1,0,0,0,3,2


In [5]:
anomalous_df.head(3)

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,fp_length,enc_comm,enc_parent_comm
0,NaN,998,0,0,-1,0,0,0,1,21
1,0.044,998,0,0,-1,0,0,0,1,21
2,-0.175,42,0,0,-1,0,0,0,2,21


In [6]:
def preprocess_suid_col(df):
    suid_list = df['suid']
    suid_new = [0 if suid_list[i] == 0 else 1 for i in range(len(suid_list))]
    df['suid'] = suid_new

In [7]:
def preprocess_timedelta(df):
    l = df['timedelta']
    l_new = [abs(x) if not pd.isna(x) else -1 for x in l]
    df['timedelta'] = l_new

In [8]:
for df in df_list:
    preprocess_suid_col(df)
    preprocess_timedelta(df)

In [9]:
df_list[0].head()

,severity_scores,timedelta,suid,cap_fp,exit,n_dang,n_dang_no_cron,fp_length,col_0,col_1,...,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,enc_comm,enc_parent_comm
0,1.0,0.0,1,0,0,2,0,92,0,0,...,0,0,0,0,1,0,0,0,1,39
1,1.0,0.0,0,0,0,0,0,38,0,0,...,0,0,0,0,1,0,0,0,2,82
2,1.0,0.0,0,0,0,0,0,28,0,0,...,0,0,0,0,1,0,0,0,3,2
3,1.0,0.0,0,0,0,0,0,28,0,0,...,0,0,0,0,1,0,0,0,4,2
4,1.0,0.0,0,0,0,0,0,21,0,0,...,0,0,0,0,1,0,0,0,5,2


In [10]:
preprocess_suid_col(anomalous_df)
preprocess_timedelta(anomalous_df)

In [11]:
anomalous_df.head()

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,fp_length,enc_comm,enc_parent_comm
0,-1.000,1,0,0,-1,0,0,0,1,21
1,0.044,1,0,0,-1,0,0,0,1,21
2,0.175,1,0,0,-1,0,0,0,2,21
3,0.129,1,0,0,-1,0,0,0,3,21
4,0.124,1,0,0,-1,0,0,0,4,21


In [12]:
columns_for_laurel = anomalous_df.columns
anomalous_dataset_encoded = anomalous_df

In [13]:
normal_dataset = df_list[0]

for i in range(1,len(df_list)):
    normal_dataset = np.vstack([normal_dataset, df_list[i]])
    
print(normal_dataset)
normal_df_encoded = pd.DataFrame(normal_dataset, columns=df_list[0].columns)
normal_df_encoded.head()

[[ 1.  0.  1. ...  0.  1. 39.]
 [ 1.  0.  0. ...  0.  2. 82.]
 [ 1.  0.  0. ...  0.  3.  2.]
 ...
 [ 1.  0.  0. ...  0.  8.  8.]
 [ 1.  0.  0. ...  0.  8. 97.]
 [ 1.  0.  0. ...  0. 20.  8.]]


,severity_scores,timedelta,suid,cap_fp,exit,n_dang,n_dang_no_cron,fp_length,col_0,col_1,...,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,enc_comm,enc_parent_comm
0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,92.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,39.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,82.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,2.0


In [14]:
columns_for_laurel

Index(['timedelta', 'suid', 'cap_fp', 'exit', 'log key', 'n_dang',
       'n_dang_no_cron', 'fp_length', 'enc_comm', 'enc_parent_comm'],
      dtype='object')

In [15]:
normal_dataset_df = pd.DataFrame(normal_df_encoded, columns=columns_for_laurel)
normal_dataset_df.fillna(-1, inplace=True)
anomalous_dataset_df = pd.DataFrame(anomalous_dataset_encoded, columns=columns_for_laurel)
#anomalous_dataset_df.fillna(0, inplace=True)

In [16]:
normal_dataset_df.head()

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,fp_length,enc_comm,enc_parent_comm
0,0.0,1.0,0.0,0.0,-1.0,2.0,0.0,92.0,1.0,39.0
1,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,38.0,2.0,82.0
2,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,28.0,3.0,2.0
3,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,28.0,4.0,2.0
4,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,21.0,5.0,2.0


In [17]:
'''
d_an_det = DeepLearningAnomalyDetection2.DeepLearningAnomalyDetection()
true_anomalies, reconstructed_anomalies, fp_ = d_an_det.train_test_model(normal_dataset_df, anomalous_dataset_df, 'autoencoder', laurel=1, plots=[0,0,1,1])
'''

"\nd_an_det = DeepLearningAnomalyDetection2.DeepLearningAnomalyDetection()\ntrue_anomalies, reconstructed_anomalies, fp_ = d_an_det.train_test_model(normal_dataset_df, anomalous_dataset_df, 'autoencoder', laurel=1, plots=[0,0,1,1])\n"

In [18]:
normal_dataset_df['enc_comm']

0        1.0
1        2.0
2        3.0
3        4.0
4        5.0
        ... 
7995     6.0
7996     7.0
7997     8.0
7998     8.0
7999    20.0
Name: enc_comm, Length: 8000, dtype: float64

In [19]:
import TimeDependentDeepLearningAnomalyDetection as td

In [20]:
detector = td.TimeDependentDeepLearningAnomalyDetection()
detector.analyze(normal_dataset_df)

Model: "CONV_LSTM_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 50, 10)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 50, 16)         │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Conv1D)           │ (None, 50, 10)         │         1,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cropping (Cropping1D)           │ (None, 10, 10)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,338 (13.04 KB)

 Trainable params: 3,338 (13.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 400499.9375
Epoch 2/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 391128.9375
Epoch 3/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 388941.2500
Epoch 4/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 388238.0312
Epoch 5/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 387648.4688
Epoch 6/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 386962.8125
Epoch 7/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 386497.8750
Epoch 8/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 385834.9688
Epoch 9/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 389604.0312
Epoch 10/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 387230.2188
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Mean Squared Error: 352963.5723762372
352963.5723762372


352963.5723762372